<a href="https://colab.research.google.com/github/ManuJLV/PracticasIA/blob/main/Proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai requests streamlit pyngrok

In [ ]:
import openai
import os
import sys
from openai import OpenAI
import time
import pandas as pd
import re
import json
from google.colab import userdata
import google.generativeai as genai
import streamlit as st
from pyngrok import ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Back-end
%%writefile /content/drive/MyDrive/Todo/IA/Generativa/Proyecto_Final/ia_core.py
import openai
import os
import sys
from openai import OpenAI
import time
import pandas as pd
import re
import json
from google.colab import userdata
import google.generativeai as genai
import streamlit as st
from pyngrok import ngrok
# --- Carga de datos y configuración de APIs ---
file_path = '/content/drive/MyDrive/Todo/IA/Generativa/Proyecto_Final/df_entrenamiento.csv'
raw_lines = open(file_path, 'r', encoding='utf-8').readlines()
header = raw_lines[0].strip().split(';')
num_cols = len(header)
data_lines = raw_lines[1:]
corrected_rows = []

for line in data_lines:
    parts = line.strip().split(';')
    if len(parts) == num_cols:
        corrected_rows.append(parts)
    elif len(parts) > num_cols:
        fixed = parts[:4]
        extra_fields = len(parts) - num_cols
        idea_index = 4
        idea_end_index = idea_index + 1 + extra_fields
        idea = ';'.join(parts[idea_index:idea_end_index])
        rest = parts[idea_end_index:]
        fixed.extend([idea] + rest)
        corrected_rows.append(fixed if len(fixed) == num_cols else ['ERROR'] * num_cols)
    else:
        corrected_rows.append(['ERROR'] * num_cols)

ejemplos_df = pd.DataFrame(corrected_rows, columns=header)
cliente = OpenAI(api_key='API_KEY_OPEN_AI')
genai.configure(api_key='API_KEY_GOOGLE_GENAI')
def evaluar_idea_con_modelos(idea, tipo_negocio, tipo_contenido, publico_objetivo, objetivo_contenido):
    # Construcción del prompt
    prompt = f"""
Eres un experto en marketing digital y redacción publicitaria. Tu tarea es evaluar una idea generada por una IA para una publicación en redes sociales. Evalúa cada criterio de forma objetiva en una escala del 0 al 5, donde:

- 0 = completamente ausente
- 1 = muy deficiente
- 2 = deficiente
- 3 = aceptable
- 4 = bueno
- 5 = excelente

### Parámetros de entrada:
- Tipo de negocio: {tipo_negocio}
- Tipo de contenido: {tipo_contenido}
- Público objetivo: {publico_objetivo}
- Objetivo del contenido: {objetivo_contenido}

### Texto generado por IA:
"{idea}"

### Criterios de evaluación:
1. **Claridad**: ¿El mensaje se comunica de forma clara, sin ambigüedades ni confusión?
2. **Alineación con el negocio**: ¿El contenido refleja adecuadamente la naturaleza del negocio?¿Está contextualizada al tipo de producto o servicio?
3. **Conexión con el público objetivo**: ¿La idea utiliza lenguaje, referencias o tono adecuados al público definido?¿Apela a intereses, emociones o necesidades del público?
4. **Cumplimiento del enfoque del contenido**: ¿La idea es coherente con el tipo de contenido indicado (ej. promocional, educativo)?¿La idea cumple con el objetivo planteado?
5. **Tono auténtico y adaptado**: ¿Se percibe una voz natural, cercana, adecuada al tipo de negocio?¿Evita frases genéricas o forzadas?
6. **Llamado a la acción (si corresponde)**: ¿Existe un llamado relevante, específico y útil para motivar acción por parte del usuario si corresponde?
7. **Originalidad y precisión**: ¿Evita frases genéricas, clichés, pleonasmos y redundancias innecesarias?

### Formato de respuesta:
Luego de puntuar cada criterio de evaluación, devuelve únicamente el puntaje global de la idea en formato JSON.
{{
  "puntaje_global": float
}}
"""

    puntaje_gpt = None
    puntaje_gemini = None

    # Evaluación con OpenAI GPT
    try:
        response_openai = cliente.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=300
        )
        evaluacion_gpt_str = response_openai.choices[0].message.content.strip()
        evaluacion_gpt = json.loads(evaluacion_gpt_str)
        puntaje_gpt = evaluacion_gpt.get("puntaje_global")
    except Exception as e:
        print(f"Error during OpenAI evaluation: {e}")

    # Evaluación con Gemini
    try:
        response_gemini = genai.generate_content(prompt)
        evaluacion_gemini_str = response_gemini.text.strip()
        # Clean up the Gemini response string to ensure it's valid JSON
        evaluacion_gemini_str = re.sub(r'```json\n', '', evaluacion_gemini_str)
        evaluacion_gemini_str = re.sub(r'\n```', '', evaluacion_gemini_str)
        evaluacion_gemini = json.loads(evaluacion_gemini_str)
        puntaje_gemini = evaluacion_gemini.get("puntaje_global")
    except Exception as e:
        print(f"Error during Gemini evaluation: {e}")

    # Cálculo del promedio si ambas evaluaciones son válidas
    if puntaje_gpt is not None and puntaje_gemini is not None:
        promedio = round((puntaje_gpt + puntaje_gemini) / 2, 2)
    else:
        promedio = None

    return promedio

def generar_idea_completa(tipo_negocio, tipo_contenido, publicos_objetivos, tipo_objetivo):
    if cliente is None:
        return "La clave de OpenAI no está configurada. No se puede generar la idea.", None

    ejemplos_positivos = ""
    ejemplos_negativos = ""

    if ejemplos_df is not None:
        ejemplos_buenos = ejemplos_df[
            (ejemplos_df["tipo_negocio"] == tipo_negocio) &
            (ejemplos_df["tipo_contenido"] == tipo_contenido) &
            (ejemplos_df["aprobado"] == True)
        ].head(2)

        for _, fila in ejemplos_buenos.iterrows():
            ejemplos_positivos += f"- \"{fila['idea']}\"\n"

        ejemplos_malos = ejemplos_df[
            (ejemplos_df["tipo_negocio"] == tipo_negocio) &
            (ejemplos_df["tipo_contenido"] == tipo_contenido) &
            (ejemplos_df["aprobado"] == False)
        ].head(2)

        for _, fila in ejemplos_malos.iterrows():
            ejemplos_negativos += f"- \"{fila['idea']}\"\n"

    prompt = f"""
    Actúa como un experto en marketing digital. Tu tarea es generar un texto publicitario breve (máx. 5 oraciones), creativo y original.
    Contexto:
      Tipo de negocio: {tipo_negocio}
      Enfoque del contenido: {tipo_contenido}
      Públicos objetivos: {publicos_objetivos}
      Objetivo del contenido: {tipo_objetivo}
    Requisitos:
      Debe ser clara, atractiva y adaptada. Sin clichés ni redundancias.
    --- Ejemplos de Ideas Exitosas ---
    {ejemplos_positivos}
    --- Ejemplos de Ideas a Evitar ---
    {ejemplos_negativos}
    Ahora, genera la nueva idea:
    """

    try:
        response = cliente.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=160,
            temperature=0.7,
        )
        idea = response.choices[0].message.content.strip()
        promedio = evaluar_idea_con_modelos(idea, tipo_negocio, tipo_contenido, publicos_objetivos, tipo_objetivo)
        return idea, promedio # Return only the idea and the average score
    except Exception as e:
        return f"Error al generar la idea: {e}", None # Return error message and None for average score

Overwriting /content/drive/MyDrive/Todo/IA/Generativa/Proyecto_Final/ia_core.py


## **APLICACION EN STREAMLIT**

In [ ]:
%%writefile app.py
# Importa la función desde tu módulo de backend ia_core.py
import openai
import os
import sys
from openai import OpenAI
import time
import pandas as pd
import re
import json
from google.colab import userdata
import google.generativeai as genai
import streamlit as st
from pyngrok import ngrok
# Add the current directory to the Python path
sys.path.append('/content/drive/MyDrive/Todo/IA/Generativa/Proyecto_Final')

from ia_core import generar_idea_completa

# --- 1. Configuración de la UI (Frontend) ---
st.set_page_config(
    page_title="Generador de Ideas IA para Redes Sociales",
    page_icon="💡",
    layout="centered",
    initial_sidebar_state="collapsed"
)

st.title("✨ Generador de Ideas IA para Redes Sociales ✨")
st.markdown("Crea ideas de contenido atractivas y validadas para tu negocio.")

# --- 2. Entradas de Usuario ---
with st.expander("Parámetros de la Idea", expanded=True):
    tipo_negocio = st.text_input("Tipo de negocio (ej: Cafetería local, Consultoría IT)", "Cafetería local")
    tipo_contenido = st.selectbox("Tipo de contenido", ["Promocional", "Educativo", "Interactivo", "Lanzamiento de Producto"], index=0)
    publicos_objetivos = st.text_input("Público(s) objetivo (ej: Jóvenes universitarios, PYMES)", "Jóvenes universitarios")
    tipo_objetivo = st.selectbox("Objetivo del contenido", ["Aumentar visitas", "Generar leads", "Construir marca", "Educación del cliente", "Aumentar ventas"], index=0)

# --- 3. Generación y Evaluación de la Idea ---
if st.button("🚀 Generar  Idea", help="Haz clic para que la IA genere y valide tu idea."):
    if not all([tipo_negocio, tipo_contenido, publicos_objetivos, tipo_objetivo]):
        st.warning("Por favor, rellena todos los campos para generar una idea.")
    else:
        with st.spinner("🧠 Pensando... ¡Generando y evaluando tu idea con IA! Esto puede tomar unos segundos..."):
            try:
                idea_generada, puntaje_promedio = generar_idea_completa(tipo_negocio, tipo_contenido, publicos_objetivos, tipo_objetivo)
                #parsear puntaje promedio a float
                #puntaje_promedio = float(puntaje_promedio)
                # Always display the generated idea if it's not None or an empty string
                if idea_generada and idea_generada != "No se pudo generar una idea.":
                    st.text_area("✨ Tu Idea de Contenido ✨", value=idea_generada, height=200, help="Puedes copiar esta idea directamente.")
                    #st.success(f"✅ ¡Idea generada y aprobada! (Puntaje promedio: ** {puntaje_promedio:.2f} **) ✨")
                    st.info("💡 Consejo: Esta idea ha pasado nuestra validación de calidad. ¡Pruébala!")

                else:
                    st.error("❌ No se logró generar una idea. Considera ajustar los parámetros o revisar los ejemplos de entrenamiento.")
                    # If idea_generada contains an error message from the backend, display it
                    if idea_generada and "Error al generar la idea:" in idea_generada:
                         st.text_area("Resultado del Intento (si hubo error)", value=idea_generada, height=100)


            except Exception as e:
                st.exception(f"Ocurrió un error inesperado durante la generación/evaluación: {e}")

# --- 4. Sección de Información y Soporte (Parte del Plan de Acceso y Soporte) ---
st.sidebar.title("Ayuda y Soporte")
st.sidebar.info(
    """
    Este generador de ideas utiliza modelos avanzados de Inteligencia Artificial para crear
    contenido publicitario.
    - **¿Dudas?** Visita nuestra [documentación](link_a_documentacion_futura).
    - **Soporte:** Contacta a soporte@tuempresa.com
    - **Acerca de:** Versión 1.0.0 (Beta)
    """
)

# --- 5. Footer (Opcional, parte de branding/información) ---
st.markdown("---")
st.markdown("Creado por Andrea Cuatindioy Ortiz y Manuel López Valencia  ¡Todos los derechos reservados 2025!")

Overwriting app.py


In [ ]:
ngrok_token = "YOUR_NGROK_AUTHTOKEN"
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
    print("Ngrok authentication token set successfully.")
else:
    raise ValueError("El authtoken de ngrok no está definido correctamente.")

Ngrok authentication token set successfully.


In [ ]:
# Asigna el token a pyngrok
os.environ["NGROK_AUTH_TOKEN"] = "2y1HagZ8t6JXkyTdInBbXnLGjRN_5AtgCHYqYmwnYYBnH5BWR"
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

# Kill any existing ngrok tunnels to avoid hitting the limit
ngrok.kill()

# Start the Streamlit app in the background
!streamlit run app.py &>/content/logs.txt &

# Connect ngrok to the Streamlit port (8501)
# Pass the address directly as the first argument
public_url = ngrok.connect("8501")
print(f"Tu app está corriendo en: {public_url}")

Tu app está corriendo en: NgrokTunnel: "https://203f-130-211-120-236.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py --server.port 8501



2025-06-10 15:43:21.566 Port 8501 is already in use
